## 2유형 [1] Motor Trend Car Road Tests

In [6]:
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import numpy as np
import pandas as pd
X_test = pd.read_csv("./datasets/Part2/mpg_X_test.csv")
X_train = pd.read_csv("./datasets/Part2/mpg_X_train.csv")
y_train = pd.read_csv("./datasets/Part2/mpg_y_train.csv")

# 사용자 코딩

### 1. 데이터 전처리
# 1) 결측치 확인
print(X_train.info())

# 2) 결측치 값은 평균값으로 채우기
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train[['horsepower']] = imputer.fit_transform( X_train[['horsepower']] )
X_test[['horsepower']] = imputer.fit_transform( X_test[['horsepower']] )

# 3) 다시 확인 (통계값)
print(X_train.describe())

# -------------------------

### 2. 모델 구축
# 컬럼명 설정
COL_DEL = ['name']
COL_NUM = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']
COL_CAT = []
COL_Y = ['isUSA']

X_train = X_train.iloc[:, 1:]
X_test = X_test.iloc[:, 1:]

# 1) 분리
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.3)

# 2) 스케일링 (StandardScaler)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])
X_tr[COL_NUM]=scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM]=scaler.transform(X_val[COL_NUM])
X_test[COL_NUM]=scaler.transform(X_test[COL_NUM])

# 3) 학습 (KNN, 의사결정나무)
from sklearn.neighbors import KNeighborsClassifier
modelKNN = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
modelKNN.fit(X_tr, y_tr.values.ravel())

from sklearn.tree import DecisionTreeClassifier
modelDT = DecisionTreeClassifier(max_depth=10)
modelDT.fit(X_tr, y_tr)

# -------------------------

### 3. 모델 평가
# 1) 예측값, 예측 확률 계산
y_val_pred = modelKNN.predict(X_val)

y_val_pred_probaKNN = modelKNN.predict_proba(X_val)
y_val_pred_probaDT = modelDT.predict_proba(X_val)

# 2) AUROC 점수 구하기
from sklearn.metrics import roc_auc_score

scoreKNN = roc_auc_score(y_val, y_val_pred_probaKNN[:, 1])
scoreDT = roc_auc_score(y_val, y_val_pred_probaDT[:, 1])

print(scoreKNN, scoreDT)

# 3) 하이퍼 파라미터 튜닝 -> 최적의 모델
best_model = None
best_score = 0

for i in range(2, 10):
    model = KNeighborsClassifier(n_neighbors=i, metric='euclidean')
    model.fit(X_tr, y_tr.values.ravel())
    y_val_pred_proba = model.predict_proba(X_val)
    score = roc_auc_score(y_val, y_val_pred_proba[:, 1])
    print(i,"개의 이웃 확인 : ", score)
    if best_score <= score:
        best_model = model

# 4) 최적 모델 도출
pred = best_model.predict_proba(X_test)[:,1]
print(pred)

# -------------------------

### 4. 제출 파일 저장
pd.DataFrame({'isUSA': pred}).to_csv('./res/003000000.csv', index=False)

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'gender': pred}).to_csv('003000000.csv', index=False)
# 제출 전에 print 문은 주석 처리